# Hyperparameter Tuning - Temel Kavramlar

Bu notebook, hyperparameter tuning tekniklerini ve uygulamalarını içerir.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.datasets import make_classification

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline


## 1. Grid Search CV

Grid Search ile tüm hyperparameter kombinasyonlarını deneme.


In [ ]:
# Veri seti
X, y = make_classification(n_samples=1000, n_features=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Grid Search parametreleri
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, None],
    'min_samples_split': [2, 5, 10]
}

# Grid Search
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print("En iyi parametreler:")
print(grid_search.best_params_)
print(f"\nEn iyi CV skoru: {grid_search.best_score_:.4f}")

# Test seti performansı
y_pred = grid_search.predict(X_test)
print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")


## 2. Random Search CV

Random Search ile daha hızlı hyperparameter arama.


In [ ]:
from scipy.stats import randint, uniform

# Random Search parametreleri
param_dist = {
    'n_estimators': randint(50, 300),
    'max_depth': randint(3, 20),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10)
}

# Random Search
random_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=50,  # 50 farklı kombinasyon dene
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

random_search.fit(X_train, y_train)

print("En iyi parametreler:")
print(random_search.best_params_)
print(f"\nEn iyi CV skoru: {random_search.best_score_:.4f}")

# Test seti performansı
y_pred = random_search.predict(X_test)
print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")


## 3. Grid Search vs Random Search Karşılaştırması


In [ ]:
comparison = pd.DataFrame({
    'Method': ['Grid Search', 'Random Search'],
    'Best CV Score': [grid_search.best_score_, random_search.best_score_],
    'Test Accuracy': [accuracy_score(y_test, grid_search.predict(X_test)), 
                      accuracy_score(y_test, random_search.predict(X_test))]
})

print("Grid Search vs Random Search:")
print(comparison.round(4))
